# Survival analysis 

In this notebook we examine whether CXCR6 expression predicts survival in human cancer patients using bulk RNA-Seq data sets publically available from the TCGA pan-cancer dataset: https://gdc.cancer.gov/about-data/publications/pancanatlas. Fig. 7 & S7 were generated using the code in this notebook. 

In [1]:
import os, sys
import pandas as pd
import numpy as np

from scipy.stats import spearmanr
from scipy.stats import f_oneway
from scipy import stats
from scipy.stats import zscore
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from collections import Counter

from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test, multivariate_logrank_test
from lifelines.plotting import add_at_risk_counts
from lifelines import CoxPHFitter
from lifelines import CoxTimeVaryingFitter

import mm_functions_14_10_2020 as mm

import itertools
import functools
from collections import defaultdict

In [2]:
# Set figure parameters 
mylines = 0.15*2.82 # the number 2.82 is the difference
					# between Illustrator 1 pt and python 1 pt.
mpl.rcParams['axes.linewidth'] = mylines # default 1
mpl.rcParams['ytick.direction'] = 'out' # default 'in'
mpl.rcParams['xtick.direction'] = 'out' # default 'in'
mpl.rcParams['xtick.major.size'] = 2 # default 4
mpl.rcParams['ytick.major.size'] = 2 # default 4
mpl.rcParams['xtick.major.width'] = mylines # default 0.5
mpl.rcParams['ytick.major.width'] = mylines # default 0.5
mpl.rcParams['grid.linewidth'] = mylines/1.5 # default 0.5
mpl.rcParams['grid.color'] = '0.8' # default 'k'
mpl.rcParams['grid.linestyle'] = 'solid'# default ':'
mpl.rcParams['legend.frameon'] = False # default True
mpl.rcParams['figure.dpi']= 300
mpl.rc("savefig", dpi=300)
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
plt.rc('font', family = 'Arial',size = 12)

## Survival functions

In [3]:
def fit_cph(end1_name = '', end1_time = None, end1_event = None, 
            end2_name = '', end2_time = None, end2_event = None,
            advice = False, strata = [], drop = [], **kwargs):
    
    # Define the keyword variables (function currently accepts up to 10 keyword variables):
    var1 = kwargs.get('var1', pd.DataFrame())
    var2 = kwargs.get('var2', pd.DataFrame())
    var3 = kwargs.get('var3', pd.DataFrame())
    var4 = kwargs.get('var4', pd.DataFrame())
    var5 = kwargs.get('var5', pd.DataFrame())
    var6 = kwargs.get('var6', pd.DataFrame())
    var7 = kwargs.get('var7', pd.DataFrame())
    var8 = kwargs.get('var8', pd.DataFrame())
    var9 = kwargs.get('var9', pd.DataFrame())
    var10 = kwargs.get('var10', pd.DataFrame())

    var1_name = kwargs.get('var1_name', str)
    var2_name = kwargs.get('var2_name', str)
    var3_name = kwargs.get('var3_name', str)
    var4_name = kwargs.get('var4_name', str)
    var5_name = kwargs.get('var5_name', str)
    var6_name = kwargs.get('var6_name', str)
    var7_name = kwargs.get('var7_name', str)
    var8_name = kwargs.get('var8_name', str)
    var9_name = kwargs.get('var9_name', str)
    var10_name = kwargs.get('var10_name', str)
    
    dummy_var1 = kwargs.get('dummy_var1', pd.DataFrame())
    dummy_var2 = kwargs.get('dummy_var2', pd.DataFrame())
    dummy_var3 = kwargs.get('dummy_var3', pd.DataFrame())
    dummy_var4 = kwargs.get('dummy_var4', pd.DataFrame())
    dummy_var5 = kwargs.get('dummy_var5', pd.DataFrame())
    
    dummy_var1_name = kwargs.get('dummy_var1_name', tuple())
    dummy_var2_name = kwargs.get('dummy_var2_name', tuple())
    dummy_var3_name = kwargs.get('dummy_var3_name', tuple())
    dummy_var4_name = kwargs.get('dummy_var4_name', tuple())
    dummy_var5_name = kwargs.get('dummy_var5_name', tuple())
    
    # Create a end_times/event dictionary: 
    end_times_dict = {end1_name : end1_time, 
                      end2_name : end2_time}
    
    end_events_dict = {end1_name : end1_event, 
                       end2_name : end2_event}
    
    # Create a variables dictionary: 
    variables = [kwargs[keyword] for keyword in kwargs.keys() if not keyword.endswith('name') if keyword.startswith('var')]
    variables_keys = [kwargs[keyword] for keyword in kwargs.keys() if keyword.endswith('name') if keyword.startswith('var')]
    variables_dict = dict(zip(variables_keys, variables)) 
  
    # Create a dummy variables dictionary: 
    dummy_variables = [kwargs[keyword] for keyword in kwargs.keys() if not keyword.endswith('name') if keyword.startswith('dummy')]
    dummy_variables_keys = [kwargs[keyword][0] for keyword in kwargs.keys() if keyword.endswith('name') if keyword.startswith('dummy')]
    dummy_variables_dict = dict(zip(dummy_variables_keys, dummy_variables))

    # Create a dummy ref dictionary: 
    dummy_ref = [kwargs[keyword][1] for keyword in kwargs.keys() if keyword.endswith('name') if keyword.startswith('dummy')]
    dummy_ref_keys = [kwargs[keyword][0] for keyword in kwargs.keys() if keyword.endswith('name') if keyword.startswith('dummy')]
    dummy_ref_dict = dict(zip(dummy_ref_keys, dummy_ref))
    
    # Add dummy variables to variables_dict: 
    dummy_columns = {}
    na = defaultdict(dict)
    for var in dummy_variables_dict.keys(): 
        
        if var not in strata: 
            
            # Safe number of NaN for each dummy variable:
            for end in [end1_name, end2_name]:
                if end:
                    na[end][var] = dummy_variables_dict[var].isna().sum()
        
            # Dropna : 
            dummy_variables_dict[var] = dummy_variables_dict[var].dropna()
        
            # Create dummies and drop ref variable: 
            temporary_df = pd.get_dummies(dummy_variables_dict[var]).drop(dummy_ref_dict[var], axis = 1)
        
            # Also initialize a list for dummy variable name storing: 
            dummy_columns[var] = list()

            for column in temporary_df.columns: 
                variables_dict[column] = temporary_df[column]
            
                # Also store dummy variable names: 
                dummy_columns[var].append(column)
        
        else:       
            variables_dict[var] = dummy_variables_dict[var]
            
            
    cph_dict = defaultdict(dict)
    # For each end, fit a cph model:
    for end in [end1_name, end2_name]: 
        if end: 
            print('Fitting:', end, '...')
        
            # For each variable, store non-NaN index values: 
            ix = {}
            for var in variables_dict.keys():
                
                ix[var] = variables_dict[var].dropna().index.values
                
                # For each patient, also store number of NaN for plotting: 
                na[end][var] = variables_dict[var].isna().sum()
            
            # Reduce dictionary of non-NaN index values to the intersecting set of all variables:  
            ix = functools.reduce(np.intersect1d, [ix[var] for var in ix.keys()])
            
            # For both end_time and end_event, store number of NaN values under ix: 
            na[end]['time'] = end_times_dict[end].loc[ix].isna().sum()
            na[end]['event'] = end_events_dict[end].loc[ix].isna().sum()
            
            # Intersect the intersecting non-NaN index of all variables with non-NaN index of end_time and end_event: 
            ix = end_times_dict[end].loc[ix].dropna().index.values
            ix = end_events_dict[end].loc[ix].dropna().index.values
            
            # For each end, store number of patients in the final intersecting ix (i.e. n): 
            n = len(ix)
            
            # For each end, store the number of events considering only patients in ix (i.e. the number of events): 
            events = end_events_dict[end].loc[ix].sum()
            print('Number of events:', events)
            
            # Create a cph_test pd.DataFrame (columns = unique variables and rows = patients): 
            end_times_dict[end].columns = [end + '_time']
            end_events_dict[end].columns = [end + '_event']
            cph_test = pd.concat([end_times_dict[end].loc[ix], end_events_dict[end].loc[ix]], axis = 1, join = 'inner', keys = [end + '_time', end + '_event'])
            
            # Join variables to pd.DataFrame() on intersecting non-NaN index: 
            for var in variables_dict.keys():
            
                variables_dict[var].name = var
                cph_test = pd.concat([cph_test, variables_dict[var].loc[ix]], axis = 1, join = 'inner') 
        
            # Check whether concatenation worked: 
            if not (cph_test.index.values == ix).all(): 
                raise Exception('Concatenated cph_test pd.DataFrame index does not match intersecting set ix')
            
            if not pd.Series(variables_keys).isin(cph_test.columns).all(): # this one should be the other way around but I leave it for now because it captures when a variable is not a pd.Series
                raise Exception('Concatenated cph_test pd.DataFrame does not contain all variables as columns')

         
            cph = CoxPHFitter()
            
            if strata:
                cph.fit(cph_test, duration_col= end + '_time', event_col= end + '_event', step_size = 0.25, strata = strata)
            
            else:
                cph.fit(cph_test, duration_col= end + '_time', event_col= end + '_event', step_size = 0.25)
                
       
            # Check the assumptions of the cph model: 
            cph.check_assumptions(cph_test, advice = advice, show_plots = advice)
            print('Concordance index:', round(cph.concordance_index_, 3))
            
            if cph.concordance_index_ < 0.5: 
                'Concordance is lower than the expected result from random predictions!'
            
            # Store cph.summary in variable cph_summary for the following operations: 
            cph_summary = cph.summary 
            
            # Change empty dummy variable NaN values from 0 into '': 
            for var in dummy_columns.keys():
                
                for key in dummy_columns[var]:
                    na[end][key] = '' # Don't delete the value as the empty string is needed to plot nothing instead of NaN 
  
                # Add dummy ref levels to cph.summary: 
                where = np.where(cph_summary.index == dummy_columns[var][0])[0][0]
                cph_summary = pd.concat([cph_summary.iloc[:where, :], pd.DataFrame({}, index = [var]), cph_summary.iloc[where:, :]])
            
            # Add stored NaN numbers to cph_summary: 
            na_concat = pd.DataFrame(pd.Series(na[end]), columns = ['number of NaN'])
            cph_summary = cph_summary.join(na_concat, how = 'left')
            
            if drop:
                cph_summary = cph_summary.drop(drop, axis = 0)
            
            # Store results in end_results_dict:
            cph_dict[end]['cph'] = cph_summary # cph fit result 
            cph_dict[end]['n'] = n # int = number of non-NaN (i.e. complete) patients
            cph_dict[end]['events'] = events 
            cph_dict[end]['na'] = na[end] # dict with keys = variable or time/event and values = number of NaN
            cph_dict[end]['ix'] = ix
            
            print('Finished fitting: ' + end + '!\n')
        
    return cph_dict

In [4]:
def plot_forest_cph(cph_summary, sort_on_pvalues = False, title = '', xlabel = '', fig_adjust = (5, 1), save = '', show = False, nan_str = True):
    
    cph_summary.index = cph_summary.index.astype(str)
    cph_summary = cph_summary.iloc[::-1]
    indexLen = np.array([len(index) for index in cph_summary.index]).max()

    a, fig, gs = mm.startfig(indexLen + cph_summary['exp(coef) upper 95%'].max() + fig_adjust[0], fig_adjust[1]*len(cph_summary['-log2(p)']))
    
    if sort_on_pvalues:
        cph_summary = cph_summary.sort_values('-log2(p)')
    
    ylabels_first = []
    ylabels_second = []

    for hazard, y, state, hazard_lower, hazard_upper, pvalue, nnan, color in zip(np.log10(cph_summary['exp(coef)']), 
                                                                                 range(0, len(cph_summary['p'])), 
                                                                                 cph_summary.index, 
                                                                                 np.log10(cph_summary['exp(coef) lower 95%']), 
                                                                                 np.log10(cph_summary['exp(coef) upper 95%']),
                                                                                 cph_summary['p'], 
                                                                                 cph_summary['number of NaN'], 
                                                                                 itertools.cycle(['lightgrey', 'white'])):
        
        if pvalue < 0.05: 
            a.scatter(hazard, y, s = 25, c = 'black', zorder = 4, marker = 's')
            a.plot([hazard_lower, hazard_upper], [y, y], c = 'black', lw = 1, zorder = 2)
        
        else:
            a.scatter(hazard, y, s = 25, c = 'grey', zorder = 4, marker = 's')
            a.plot([hazard_lower, hazard_upper], [y, y], c = 'grey', lw = 1, zorder = 2)
    
        # 10**hazard + 95% CI str 
        if np.isnan(pvalue):
            
            if nan_str: 
                hazard_str = str(nnan)
            
            else: 
                hazard_str = ''
        
        else:
            
            if nan_str:
                hazard_str = '{:.2f}'.format(round(10**hazard, 2)) + ' (' + '{:.2f}'.format(round(10**hazard_lower, 2)) + '-' + '{:.2f}'.format(round(10**hazard_upper,2)) + ') ' + '    {:.2f}'.format(round(np.log10(pvalue),2)) + ' ' + str(nnan)
                
            else:
                hazard_str = '{:.2f}'.format(round(10**hazard, 2)) + ' (' + '{:.2f}'.format(round(10**hazard_lower, 2)) + '-' + '{:.2f}'.format(round(10**hazard_upper,2)) + ') ' + '    {:.2f}'.format(round(np.log10(pvalue),2))
                
        ylabels_first.append(state)
        ylabels_second.append(hazard_str)

        a.axhspan(y - 0.5, y + 0.5, facecolor = color, clip_on = False, zorder = -2)
        #a.add_patch(patches.Rectangle((-indexLen/31, y-0.5), abs(-indexLen/31) + cph_summary['exp(coef) upper 95%'].max(), 1, alpha=1, facecolor = color, zorder = -2, clip_on=False))
            
    a.axvline(np.log10(1), zorder = 0, lw = 1, c = 'grey')
            
    # set yticks
    a.set_yticks(range(0, len(cph_summary['-log2(p)'])))
    a.set_yticklabels(ylabels_first)

    secay = a.secondary_yaxis('right')
    secay.set_yticks(range(0, len(cph_summary['-log2(p)'])))
    secay.set_yticklabels(ylabels_second)
        
    a.tick_params(axis=u'both', which=u'both',length=0)
    secay.tick_params(axis=u'both', which=u'both',length=0)

    # set spines
    a.spines['left'].set_visible(False)
    a.spines['right'].set_color('white') # matplotlib secondary yaxis bug: I can't make the right spine invisible anymore
    a.spines['top'].set_visible(False)

    # set title and xaxis label
    a.set_title(title)
    a.set_xlabel(xlabel)

    a.set_ylim(-0.5, len(cph_summary['-log2(p)'])-0.5)
    plt.tight_layout()
    plt.savefig(save, dpi = 400)
    
    if show == True: 
        plt.show()
    
    plt.close()

## Load TCGA data

In [5]:
# TCGA sample info from PANCAN atlas: https://gdc.cancer.gov/about-data/publications/pancanatlas
TCGA_sample_info = pd.read_excel('./data/tcga/TCGA-CDR-SupplementalTableS1.xlsx', sheet_name=0, header=0, index_col=1)
TCGA_survival = pd.read_excel('./data/tcga/TCGA-CDR-SupplementalTableS1.xlsx', sheet_name=2, header=0, index_col=1)

In [6]:
TCGA_sample_info['ajcc_pathologic_tumor_stage'].unique()

array(['Stage II', 'Stage IV', 'Stage III', '[Not Available]', 'Stage I',
       'Stage X', 'Stage IIB', 'Stage IA', 'Stage IIIA', 'Stage IIA',
       'Stage IIIC', 'Stage IB', 'Stage IIIB', '[Discrepancy]',
       'Stage IVB', 'Stage IVA', 'Stage IIC', '[Not Applicable]',
       '[Unknown]', 'Stage IVC', 'I/II NOS', 'Stage 0', 'IS'],
      dtype=object)

In [7]:
stages_map = {
    'Stage 0': 'Stage 0 & I', # Merge stage 0 and I 
    'Stage I': 'Stage 0 & I', 
    'Stage IA': 'Stage 0 & I', 
    'Stage IB': 'Stage 0 & I', 
    'I/II NOS': np.nan, # Ambigious 
    'Stage II': 'Stage II',
    'Stage IIA': 'Stage II', 
    'Stage IIB': 'Stage II', 
    'Stage IIC': 'Stage II', 
    'Stage III': 'Stage III', 
    'Stage IIIA': 'Stage III', 
    'Stage IIIB': 'Stage III', 
    'Stage IIIC': 'Stage III', 
    'Stage IV': 'Stage IV',
    'Stage IVA': 'Stage IV', 
    'Stage IVB': 'Stage IV', 
    'Stage IVC': 'Stage IV', 
    'Stage X': np.nan, 
    '[Not Available]': np.nan, 
    '[Not Applicable]': np.nan,
    '[Discrepancy]': np.nan,
    'IS': np.nan, 
    '[Unknown]': np.nan 
}

gender_map = {
    'MALE': 0, 
    'FEMALE': 1 
}

In [8]:
TCGA_sample_info['tumor_stage'] = TCGA_sample_info['ajcc_pathologic_tumor_stage'].map(stages_map)
TCGA_sample_info['sex'] = TCGA_sample_info['gender'].map(gender_map)

In [9]:
# Load SKCM additional sample info
SKCM_info = pd.read_excel('/Users/mariusmessemaker/Documents/Project/mempel/data/tcga/SKCM.xlsx', sheet_name = 3, header = 1, index_col = 0)
SKCM_info.index = SKCM_info.index.str[0:12]
print(SKCM_info.shape)
SKCM_info = SKCM_info.iloc[~SKCM_info.index.duplicated(), :]
print(SKCM_info.shape)

regional_vs_primary_SKCM_map = {
    '-' : np.nan, 
    'Primary_Disease' : 'Primary_Disease', 
    'Regional_Lymph_Node' : 'Regional_Lymph_Node', 
    'Regional_Skin_or_Soft_Tissue' : 'Regional_Skin_or_Soft_Tissue', 
    'Distant_Metastases': 'Distant_Metastases'
}

print(SKCM_info['REGIONAL_VS_PRIMARY'].value_counts())
SKCM_info['REGIONAL_VS_PRIMARY'] = SKCM_info['REGIONAL_VS_PRIMARY'].map(regional_vs_primary_SKCM_map)
print(SKCM_info['REGIONAL_VS_PRIMARY'].value_counts())
print(SKCM_info['REGIONAL_VS_PRIMARY'].unique())

(333, 111)
(331, 111)
Regional_Lymph_Node             160
Regional_Skin_or_Soft_Tissue     52
-                                43
Primary_Disease                  41
Distant_Metastases               35
Name: REGIONAL_VS_PRIMARY, dtype: int64
Regional_Lymph_Node             160
Regional_Skin_or_Soft_Tissue     52
Primary_Disease                  41
Distant_Metastases               35
Name: REGIONAL_VS_PRIMARY, dtype: int64
[nan 'Primary_Disease' 'Regional_Lymph_Node'
 'Regional_Skin_or_Soft_Tissue' 'Distant_Metastases']


In [10]:
# TCGA pancan expression data
# gene expression from pancan atlas: https://gdc.cancer.gov/about-data/publications/pancanatlas
TCGA_pancan_exp = pd.read_csv('./data/tcga/EBPlusPlusAdjustPANCAN_IlluminaHiSeq_RNASeqV2.geneExp.tsv', sep='\t', header=0, index_col=0)

# sample codes from code table https://gdc.cancer.gov/resources-tcga-users/tcga-code-tables/sample-type-codes
tumor_codes = ['01', '02', '05', '06'] # 06 is also metastatic sites 
normal_codes = ['10', '11', '12', '13', '14']

# divide samples into tumors and normals
stype = ['tumor' if x.split('-')[3][0:2] in tumor_codes else 'normal' if x.split('-')[3][0:2] in normal_codes else 'NA' for x in TCGA_pancan_exp.columns.values]

sample_type = pd.Series(stype, index = TCGA_pancan_exp.columns.values)

#change index to gene names only
TCGA_pancan_exp.index = map(lambda x: x.split('|')[0], TCGA_pancan_exp.index.values)

# extract tumors only
tumor_exp = TCGA_pancan_exp[sample_type[sample_type=='tumor'].index.values]

# extract normal samples only
normal_exp = TCGA_pancan_exp[sample_type[sample_type=='normal'].index.values]

# shorten column names/patient IDs
tumor_exp.rename(columns = lambda x: x[0:12], inplace=True)
normal_exp.rename(columns = lambda x: x[0:12], inplace=True)

# remove duplicate gene expression data for the same sample
tumor_exp_unique = tumor_exp.iloc[:, ~tumor_exp.columns.duplicated()].T
normal_exp_unique = normal_exp.iloc[:, ~normal_exp.columns.duplicated()].T

/Users/mariusmessemaker/miniconda3/envs/py36-sc/lib/python3.6/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [11]:
# Transform expression matrix to dense ranking for DC gene scoring 
tumor_dense_rank_unique = tumor_exp_unique.transform('rank', method = 'dense', axis = 0)

## Score genes

In [12]:
state_genes = {
'CD8B': ['CD8B'],
'CCR1': ['CCR1'],
'CCR10': ['CCR10'], 
'CCR2': ['CCR2'], 
'CCR3': ['CCR3'], 
'CCR4': ['CCR4'], 
'CCR5': ['CCR5'], 
'CCR6':['CCR6'], 
'CCR7':['CCR7'], 
'CCR8':['CCR8'], 
'CCR9':['CCR9'], 
'CX3CR1':['CX3CR1'],
'CXCR1':['CXCR1'], 
'CXCR2':['CXCR2'],
'CXCR3':['CXCR3'],
'CXCR4':['CXCR4'],
'CXCR5':['CXCR5'],
'CXCR6':['CXCR6'], 
'CXCR7':['CXCR7'],  
'XCR1':['XCR1'],
'CD19':['CD19'],
'CD4':['CD4'],
'CXCL16':['CXCL16'],
'IL12B':['IL12B'],
'NK cells':['NCR1', 'SH2D1B']} 

In [13]:
cancers = ['BRCA', 'HNSC', 'LUAD', 'LUSC', 'STAD', 'KIRC', 'BLCA', 'LIHC', 'SKCM']

In [14]:
log_zscore = True

state_scores = defaultdict(dict)
for cancer in cancers:
    print(cancer)
    os.mkdir('./outputs/survival/' + cancer)
    os.mkdir('./outputs/survival/' + cancer + '/distribution/')
    
    # Exclude samples that are metastatic (or lymph-node metastatic in SKCM)
    #if cancer in exclude_dict.keys():
     #   ix0 = TCGA_sample_info[~TCGA_sample_info.index.isin(exclude_dict[cancer])].index.values        
      #  ix1 = TCGA_sample_info[TCGA_sample_info['type']==cancer].index.values 
       #  ix = np.intersect1d(ix0, ix1)
    
    #elif cancer == 'SKCM': 
     #   ix = SKCM_info[~SKCM_info.index.isin(SKCM_ix)].index.values 
        
    ix = TCGA_sample_info[TCGA_sample_info['type']==cancer].index.values
    
    # Check whether indexing worked: 
    print(TCGA_sample_info.loc[ix]['type'].unique(), 'n=', len([sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_exp_unique.index]))
    
    for state in state_genes.keys():
        print('Started scoring:', state, '...')
        
        # Check whether genes are detected 
        genes_first = [gene for gene in state_genes[state] if gene in tumor_exp_unique.columns]
        genes = []
        
        for gene in genes_first:
            
            # Check whether genes are NaN although they are detected 
            if tumor_exp_unique.loc[[sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_exp_unique.index], gene].isna().sum() < 1:
                genes.append(gene)
                
            else:
                # If a gene is not included in the columns or NaN: 
                print('Not detected:', gene)
        
        # Plot distribution before gene scoring 
        sns.displot(tumor_exp_unique.loc[[sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_exp_unique.index], genes].mean(1))
        plt.title('Distribution ' + state + ' before scoring')
        plt.savefig('./outputs/survival/' + cancer + '/distribution/' + state + '_before.pdf', dpi = 300)
        plt.close()
        
        # Score genes 
        if log_zscore:
            # EdgeR pseudo count calculation https://bioinformatics.stackexchange.com/questions/4985/how-is-prior-count-used-by-edgers-cpm
            pseudo = (tumor_exp_unique.loc[[sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_exp_unique.index]].sum(1)/tumor_exp_unique.loc[[sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_exp_unique.index]].sum(1).mean())*2*0.25
            
            if not (pseudo.index == tumor_exp_unique.loc[[sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_exp_unique.index]].index).all():
                raise Exception('pseudo index not the same as expression matrix index')
            
            if len(genes) <= 1:
                state_scores[cancer][state] = np.log2(tumor_exp_unique.loc[[sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_exp_unique.index], 
                                                        genes].add(pseudo, axis = 0)).apply(zscore, axis = 0)[genes[0]]
            else:
                state_scores[cancer][state] = np.log2(tumor_exp_unique.loc[[sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_exp_unique.index], 
                                                      genes].add(pseudo, axis = 0)).apply(zscore, axis = 0).mean(1)
                                            
        else: 
            state_scores[cancer][state] = tumor_dense_rank_unique.loc[[sample for sample in TCGA_sample_info.loc[ix].index.values if sample in tumor_dense_rank_unique.index], 
                                                                       genes].sum(1)/len(genes)
            state_scores[cancer][state] = state_scores[cancer][state]/state_scores[cancer][state].max()
            
        print(len(state_scores[cancer][state]))
        
        # Plot distribution of scored genes 
        sns.displot(state_scores[cancer][state])
        plt.title('Distribution ' + state + ' after scoring')
        plt.savefig('./outputs/survival/' + cancer + '/distribution/' + state + '_after.pdf', dpi = 300)
        plt.close()

BRCA
['BRCA'] n= 1095
Started scoring: CD8B ...
1095
Started scoring: CCR1 ...
1095
Started scoring: CCR10 ...
1095
Started scoring: CCR2 ...
1095
Started scoring: CCR3 ...
1095
Started scoring: CCR4 ...
1095
Started scoring: CCR5 ...
1095
Started scoring: CCR6 ...
1095
Started scoring: CCR7 ...
1095
Started scoring: CCR8 ...
1095
Started scoring: CCR9 ...
1095
Started scoring: CX3CR1 ...
1095
Started scoring: CXCR1 ...
1095
Started scoring: CXCR2 ...
1095
Started scoring: CXCR3 ...
1095
Started scoring: CXCR4 ...
1095
Started scoring: CXCR5 ...
1095
Started scoring: CXCR6 ...
1095
Started scoring: CXCR7 ...
1095
Started scoring: XCR1 ...
1095
Started scoring: CD19 ...
1095
Started scoring: CD4 ...
1095
Started scoring: CXCL16 ...
1095
Started scoring: IL12B ...
1095
Started scoring: NK cells ...
1095
HNSC
['HNSC'] n= 520
Started scoring: CD8B ...
520
Started scoring: CCR1 ...
520
Started scoring: CCR10 ...
520
Started scoring: CCR2 ...
520
Started scoring: CCR3 ...
520
Started scoring

## Continious association of scores

In [15]:
states = ['CD8B', 'CXCL16', 'CXCR6', 'CXCR3', 'CXCR2', 'CCR5', 'CCR10', 'CCR8', 'CD19', 'CD4', 'NK cells', 'IL12B'] 

In [16]:
for cancer in cancers: 
    print(cancer)
    os.mkdir('./outputs/survival/' + cancer + '/scatter/')
    
    # Compare all combinations: 
    for state1, state2 in itertools.combinations(states, 2):
        name = state1 + '_with_' + state2
        print('Comparing:', name)
        
        # Plot scatterplot that shows relation of the continious variables: 
        a, fig, gs = mm.startfig(8, 8) 
        
        if not (state_scores[cancer][state1].index == state_scores[cancer][state2].index).all():
            raise Excpetion('Indexes are not the same')
        
        a.scatter(state_scores[cancer][state1], state_scores[cancer][state2], c = 'black', s = 5)
        a.set_xlabel(state1 + ' score')
        a.set_ylabel(state2 + ' score')
        
        if spearmanr(state_scores[cancer][state1], state_scores[cancer][state2])[1] < 0.001:
            a.set_title(cancer + ' r='+str(round(spearmanr(state_scores[cancer][state1], state_scores[cancer][state2])[0],2)) + ' p < 0.001')
        else:
            a.set_title(cancer + ' r='+str(round(spearmanr(state_scores[cancer][state1], state_scores[cancer][state2])[0],2)) + ' p=' + str(round(spearmanr(state_scores[cancer][state1], state_scores[cancer][state2])[1],3)))
            
        plt.locator_params(nbins=10)
        plt.tight_layout()
        plt.savefig('./outputs/survival/' + cancer + '/scatter/' + name + '.pdf')
        plt.close()

BRCA
Comparing: CD8B_with_CXCL16
Comparing: CD8B_with_CXCR6
Comparing: CD8B_with_CXCR3
Comparing: CD8B_with_CXCR2
Comparing: CD8B_with_CCR5
Comparing: CD8B_with_CCR10
Comparing: CD8B_with_CCR8
Comparing: CD8B_with_CD19
Comparing: CD8B_with_CD4
Comparing: CD8B_with_NK cells
Comparing: CD8B_with_IL12B
Comparing: CXCL16_with_CXCR6
Comparing: CXCL16_with_CXCR3
Comparing: CXCL16_with_CXCR2
Comparing: CXCL16_with_CCR5
Comparing: CXCL16_with_CCR10
Comparing: CXCL16_with_CCR8
Comparing: CXCL16_with_CD19
Comparing: CXCL16_with_CD4
Comparing: CXCL16_with_NK cells
Comparing: CXCL16_with_IL12B
Comparing: CXCR6_with_CXCR3
Comparing: CXCR6_with_CXCR2
Comparing: CXCR6_with_CCR5
Comparing: CXCR6_with_CCR10
Comparing: CXCR6_with_CCR8
Comparing: CXCR6_with_CD19
Comparing: CXCR6_with_CD4
Comparing: CXCR6_with_NK cells
Comparing: CXCR6_with_IL12B
Comparing: CXCR3_with_CXCR2
Comparing: CXCR3_with_CCR5
Comparing: CXCR3_with_CCR10
Comparing: CXCR3_with_CCR8
Comparing: CXCR3_with_CD19
Comparing: CXCR3_with_CD

## Association of scores with tumor stage 

In [17]:
states = ['CD8B', 'CXCL16', 'CXCR6', 'CXCR3', 'CXCR2', 'CCR5', 'CCR10', 'CD4', 'NK cells', 'IL12B'] 

In [18]:
for cancer in cancers: 
    print(cancer)
    os.mkdir('./outputs/survival/' + cancer + '/tumor_stage_boxplot/')
    
    for state in states: 
        print(state)

        score_stage_df = pd.DataFrame(state_scores[cancer][state], columns = [state]).join(TCGA_sample_info['tumor_stage']).dropna()

        kruskal = stats.kruskal(score_stage_df[score_stage_df['tumor_stage'] == 'Stage 0 & I'][state],
                              score_stage_df[score_stage_df['tumor_stage'] == 'Stage II'][state],
                              score_stage_df[score_stage_df['tumor_stage'] == 'Stage III'][state], 
                              score_stage_df[score_stage_df['tumor_stage'] == 'Stage IV'][state])
        
        if kruskal[1] < 0.05: 
            print('significant!')
        
        order = ['Stage 0 & I', 'Stage II', 'Stage III', 'Stage IV']
        ax = sns.swarmplot(x = 'tumor_stage', y = state, data = score_stage_df, color = 'black', zorder = 0, s = 3, order = order)
        ax = sns.boxplot(x = 'tumor_stage', y = state, data = score_stage_df, color = 'white', zorder = 4, order = order)
        
        ax.set_xticklabels(['Stage 0 & I (n=' + str(len(score_stage_df[score_stage_df['tumor_stage'] == 'Stage 0 & I'])) + ')',
                            'Stage II (n=' + str(len(score_stage_df[score_stage_df['tumor_stage'] == 'Stage II'])) + ')',
                            'Stage III (n=' + str(len(score_stage_df[score_stage_df['tumor_stage'] == 'Stage III'])) + ')',
                            'Stage IV (n=' + str(len(score_stage_df[score_stage_df['tumor_stage'] == 'Stage IV'])) + ')'])
        
        plt.xlabel('')
        
        if kruskal[1] < 0.001:
            ax.set_title(cancer + ' H=' + str(round(kruskal[0], 2)) + ' p < 0.001')
        else: 
            ax.set_title(cancer + ' H=' + str(round(kruskal[0], 2)) + ' p=' + str(round(kruskal[1], 2)))
            
        for patch in ax.artists:
            r, g, b, a = patch.get_facecolor()
            patch.set_facecolor((r, g, b, 0))
        
        plt.ylabel(state + ' score')
        plt.tight_layout()
        plt.savefig('./outputs/survival/' + cancer + '/' + 'tumor_stage_boxplot/' + state + '.pdf')
        plt.close()

BRCA
CD8B


/Users/mariusmessemaker/miniconda3/envs/py36-sc/lib/python3.6/site-packages/seaborn/categorical.py:1296: UserWarning: 5.5% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


CXCL16


/Users/mariusmessemaker/miniconda3/envs/py36-sc/lib/python3.6/site-packages/seaborn/categorical.py:1296: UserWarning: 13.2% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


CXCR6
CXCR3


...

CCR10
significant!


/Users/mariusmessemaker/miniconda3/envs/py36-sc/lib/python3.6/site-packages/seaborn/categorical.py:1296: UserWarning: 9.2% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


CD4
NK cells


/Users/mariusmessemaker/miniconda3/envs/py36-sc/lib/python3.6/site-packages/seaborn/categorical.py:1296: UserWarning: 12.6% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


IL12B
HNSC
CD8B
CXCL16
CXCR6
significant!
CXCR3
significant!
CXCR2
significant!
CCR5
significant!
CCR10
CD4
NK cells
significant!
IL12B
LUAD
CD8B
significant!
CXCL16
CXCR6
significant!
CXCR3
CXCR2
CCR5
significant!
CCR10
significant!
CD4
significant!
NK cells
IL12B
significant!
LUSC
CD8B
CXCL16
CXCR6
CXCR3
CXCR2
CCR5
CCR10
CD4
NK cells
IL12B
STAD
CD8B
CXCL16
CXCR6
CXCR3
significant!
CXCR2
significant!
CCR5
significant!
CCR10
significant!
CD4
significant!
NK cells
IL12B
KIRC
CD8B
significant!
CXCL16
CXCR6
significant!
CXCR3
significant!
CXCR2
CCR5
significant!
CCR10
CD4
NK cells
IL12B
BLCA
CD8B
CXCL16
CXCR6
CXCR3
CXCR2
CCR5
CCR10
CD4
NK cells
IL12B
LIHC
CD8B
CXCL16
CXCR6
CXCR3
CXCR2
CCR5
CCR10
CD4
NK cells
IL12B
SKCM
CD8B
significant!
CXCL16
CXCR6
significant!
CXCR3
significant!
CXCR2
CCR5
significant!
CCR10
CD4
significant!
NK cells
significant!
IL12B


## Association scores with lymphocyte score in SKCM

In [19]:
states = ['CD8B', 'CXCL16', 'CXCR6', 'CXCR3', 'CXCR2', 'CCR5', 'CCR10', 'CD4', 'NK cells', 'IL12B'] 

In [20]:
lymphocyte_score_map = {
    0: 'Lc score 0 & 2', 
    2: 'Lc score 0 & 2', 
    3: 'Lc score 3 & 4', 
    4: 'Lc score 3 & 4',
    5: 'Lc score 5 & 6',
    6: 'Lc score 5 & 6', 
    '-': np.nan
}

In [21]:
SKCM_info['Lc_score'] = SKCM_info['LYMPHOCYTE.SCORE'].map(lymphocyte_score_map)

In [22]:
cancer = 'SKCM'
os.mkdir('./outputs/survival/' + cancer + '/lc_score_boxplot/')
for state in states: 
    print(state)

    score_stage_df = pd.DataFrame(state_scores[cancer][state], columns = [state]).join(SKCM_info['Lc_score']).dropna()

    kruskal = stats.kruskal(score_stage_df[score_stage_df['Lc_score'] == 'Lc score 0 & 2'][state],
                          score_stage_df[score_stage_df['Lc_score'] == 'Lc score 3 & 4'][state],
                          score_stage_df[score_stage_df['Lc_score'] == 'Lc score 5 & 6'][state])
  
        
    if kruskal[1] < 0.05: 
        print('significant!')
        
    order = ['Lc score 0 & 2', 'Lc score 3 & 4', 'Lc score 5 & 6']
    ax = sns.swarmplot(x = 'Lc_score', y = state, data = score_stage_df, color = 'black', zorder = 0, s = 6, order = order)
    ax = sns.boxplot(x = 'Lc_score', y = state, data = score_stage_df, color = 'white', zorder = 4, order = order)
    
    ax.set_xticklabels(['Lc score 0 & 2 (n=' + str(len(score_stage_df[score_stage_df['Lc_score'] == 'Lc score 0 & 2'])) + ')',
                        'Lc score 3 & 4 (n=' + str(len(score_stage_df[score_stage_df['Lc_score'] == 'Lc score 3 & 4'])) + ')',
                        'Lc score 5 & 6 (n=' + str(len(score_stage_df[score_stage_df['Lc_score'] == 'Lc score 5 & 6'])) + ')'])
    plt.xlabel('')
               
    if kruskal[1] < 0.001:
        ax.set_title(cancer + ' H=' + str(round(kruskal[0], 2)) + ' p < 0.001')
    else: 
        ax.set_title(cancer + ' H=' + str(round(kruskal[0], 2)) + ' p=' + str(round(kruskal[1], 3)))
            
    for patch in ax.artists:
        r, g, b, a = patch.get_facecolor()
        patch.set_facecolor((r, g, b, 0))
        
    plt.ylabel(state + ' score')
    plt.tight_layout()
    plt.savefig('./outputs/survival/' + cancer + '/' + 'lc_score_boxplot/' + state + '.pdf')
    plt.close()

CD8B
significant!
CXCL16
significant!
CXCR6
significant!
CXCR3
significant!
CXCR2
significant!
CCR5
significant!
CCR10
CD4
significant!
NK cells
significant!
IL12B
significant!


/Users/mariusmessemaker/miniconda3/envs/py36-sc/lib/python3.6/site-packages/seaborn/categorical.py:1296: UserWarning: 12.0% of the points cannot be placed; you may want to decrease the size of the markers or use stripplot.
  warnings.warn(msg, UserWarning)


## Univariate CPH

In [23]:
states = ['CCR1', 'CCR10', 'CCR2', 'CCR3', 'CCR4', 'CCR5', 'CCR6','CCR7', 'CCR8', 
          'CCR9', 'CX3CR1', 'CXCR1', 'CXCR2', 'CXCR3', 'CXCR4', 'CXCR5', 'CXCR6', 'CXCR7', 'XCR1'] 

In [24]:
for cancer in cancers:
    print(cancer)
    os.mkdir('./outputs/survival/' + cancer + '/cph/')
    
    cph_concat = {}
    cph_concat['OS'] = pd.DataFrame()
    cph_concat['PFS'] = pd.DataFrame()
    
    for state in states: 
        print(state)
        ix = state_scores[cancer][state].index.values
        
        cph_dict = fit_cph(end1_name = 'OS', end1_time = TCGA_sample_info[TCGA_sample_info['type']==cancer].loc[ix,'OS.time'], end1_event = TCGA_sample_info[TCGA_sample_info['type']==cancer].loc[ix, 'OS'], 
                           end2_name = 'PFS', end2_time = TCGA_survival[TCGA_survival['type']==cancer].loc[ix, 'PFS.time'], end2_event = TCGA_survival[TCGA_survival['type']==cancer].loc[ix, 'PFS'], 
                           var1 = state_scores[cancer][state], var1_name = state)
        
        for end in cph_dict.keys(): 
            
            cph_concat[end] = pd.concat([cph_concat[end], cph_dict[end]['cph']])
            
    
    for end in cph_concat.keys():
        print(end)
        
        
        plot_forest_cph(cph_concat[end], title = cancer + ' ' + end, xlabel = 'log10(HR) (95% CI) (n=' + str(cph_dict[end]['n']) + ')', sort_on_pvalues = True, nan_str = False,   
                        fig_adjust = (5, 1), save = './outputs/survival/' + cancer + '/cph/' + end +'_chemokine_receptors.pdf')
    
        print(cph_dict[end]['na'])
    print('\n')
    print('\n')

BRCA
CCR1
Fitting: OS ...
Number of events: 151.0
Proportional hazard assumption looks okay.
Concordance index: 0.492
Finished fitting: OS!

Fitting: PFS ...
Number of events: 204.0
Proportional hazard assumption looks okay.
Concordance index: 0.495
Finished fitting: PFS!

CCR10
Fitting: OS ...
Number of events: 151.0
Proportional hazard assumption looks okay.
Concordance index: 0.524
Finished fitting: OS!

Fitting: PFS ...
Number of events: 204.0
Proportional hazard assumption looks okay.
Concordance index: 0.5
Finished fitting: PFS!

CCR2
Fitting: OS ...
Number of events: 151.0
Proportional hazard assumption looks okay.
Concordance index: 0.56
Finished fitting: OS!

Fitting: PFS ...
Number of events: 204.0
Proportional hazard assumption looks okay.
Concordance index: 0.556
Finished fitting: PFS!

CCR3
Fitting: OS ...
Number of events: 151.0
Proportional hazard assumption looks okay.
Concordance index: 0.518
Finished fitting: OS!

Fitting: PFS ...
Number of events: 204.0
Proportional 

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1094 total observations, 943 right-censored observations>
         test_name = proportional_hazard_test

---
           test_statistic    p  -log2(p)
CCR6 km              6.62 0.01      6.63
     rank            4.92 0.03      5.23



1. Variable 'CCR6' failed the non-proportional test: p-value is 0.0101.

Concordance index: 0.56
Finished fitting: OS!

Fitting: PFS ...
Number of events: 204.0


<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1094 total observations, 890 right-censored observations>
         test_name = proportional_hazard_test

---
           test_statistic      p  -log2(p)
CCR6 km              9.93 <0.005      9.27
     rank            6.90   0.01      6.86



1. Variable 'CCR6' failed the non-proportional test: p-value is 0.0016.

Concordance index: 0.578
Finished fitting: PFS!

CCR7
Fitting: OS ...
Number of events: 151.0
Proportional hazard assumption looks okay.
Concordance index: 0.554
Finished fitting: OS!

Fitting: PFS ...
Number of events: 204.0
Proportional hazard assumption looks okay.
Concordance index: 0.562
Finished fitting: PFS!

CCR8
Fitting: OS ...
Number of events: 151.0
Proportional hazard assumption looks okay.
Concordance index: 0.501
Finished fitting: OS!

Fitting: PFS ...
Number of events: 204.0
Proportional hazard assumption looks okay.
Concordance index: 0.504
Finished fitting: PFS!

CCR9
Fitting: OS ...
Number of events: 151.0
Proportional hazard assumption looks okay.
Concordance index: 0.562
Finished fitting: OS!

Fitting: PFS ...
Number of events: 204.0
Proportional hazard assumption looks okay.
Concordance index: 0.556
Finished fitting: PFS!

CX3CR1
Fitting: OS ...
Number of events: 151.0
Proportional hazard as

...



1. Variable 'CXCR5' failed the non-proportional test: p-value is 0.0028.

Concordance index: 0.568
Finished fitting: PFS!

CXCR6
Fitting: OS ...
Number of events: 183.0
Proportional hazard assumption looks okay.
Concordance index: 0.565
Finished fitting: OS!

Fitting: PFS ...
Number of events: 259.0
Proportional hazard assumption looks okay.
Concordance index: 0.556
Finished fitting: PFS!

CXCR7
Fitting: OS ...
Number of events: 183.0
Proportional hazard assumption looks okay.
Concordance index: 0.52
Finished fitting: OS!

Fitting: PFS ...
Number of events: 259.0
Proportional hazard assumption looks okay.
Concordance index: 0.547
Finished fitting: PFS!

XCR1
Fitting: OS ...
Number of events: 183.0
Proportional hazard assumption looks okay.
Concordance index: 0.595
Finished fitting: OS!

Fitting: PFS ...
Number of events: 259.0
Proportional hazard assumption looks okay.
Concordance index: 0.571
Finished fitting: PFS!

OS
{'XCR1': 0, 'time': 9, 'event': 0}
PFS
{'XCR1': 0, 'time': 9, 'e

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 496 total observations, 244 right-censored observations>
         test_name = proportional_hazard_test

---
             test_statistic    p  -log2(p)
CX3CR1 km              6.46 0.01      6.50
       rank            6.32 0.01      6.39



1. Variable 'CX3CR1' failed the non-proportional test: p-value is 0.0110.

Concordance index: 0.493
Finished fitting: PFS!

CXCR1
Fitting: OS ...
Number of events: 212.0
Proportional hazard assumption looks okay.
Concordance index: 0.551
Finished fitting: OS!

Fitting: PFS ...
Number of events: 252.0
Proportional hazard assumption looks okay.
Concordance index: 0.535
Finished fitting: PFS!

CXCR2
Fitting: OS ...
Number of events: 212.0
Proportional hazard assumption looks okay.
Concordance index: 0.525
Finished fitting: OS!

Fitting: PFS ...
Number of events: 252.0
Proportional hazard assumption looks okay.
Concordance index: 0.513
Finished fitting: PFS!

CXCR3
Fitting: OS ...
Number of events: 212.0
Proportional hazard assumption looks okay.
Concordance index: 0.496
Finished fitting: OS!

Fitting: PFS ...
Number of events: 252.0
Proportional hazard assumption looks okay.
Concordance index: 0.511
Finished fitting: PFS!

CXCR4
Fitting: OS ...
Number of events: 212.0
Proportional hazar

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 408 total observations, 179 right-censored observations>
         test_name = proportional_hazard_test

---
            test_statistic    p  -log2(p)
CXCR2 km              6.64 0.01      6.65
      rank            7.16 0.01      7.07



1. Variable 'CXCR2' failed the non-proportional test: p-value is 0.0075.

Concordance index: 0.532
Finished fitting: PFS!

CXCR3
Fitting: OS ...
Number of events: 178.0
Proportional hazard assumption looks okay.
Concordance index: 0.541
Finished fitting: OS!

Fitting: PFS ...
Number of events: 229.0
Proportional hazard assumption looks okay.
Concordance index: 0.533
Finished fitting: PFS!

CXCR4
Fitting: OS ...
Number of events: 178.0
Proportional hazard assumption looks okay.
Concordance index: 0.494
Finished fitting: OS!

Fitting: PFS ...
Number of events: 229.0
Proportional hazard assumption looks okay.
Concordance index: 0.501
Finished fitting: PFS!

CXCR5
Fitting: OS ...
Number of events: 178.0
Proportional hazard assumption looks okay.
Concordance index: 0.52
Finished fitting: OS!

Fitting: PFS ...
Number of events: 229.0
Proportional hazard assumption looks okay.
Concordance index: 0.522
Finished fitting: PFS!

CXCR6
Fitting: OS ...
Number of events: 178.0
Proportional hazard 

## All cancers multivariate CPH strafified by age and covariate = sex 

In [25]:
states = ['CD8B', 'CXCR6', 'CXCR3', 'CXCR2', 'CCR5', 'CCR10', 'IL12B', 'CD19', 'NK cells', 'CXCL16']

In [26]:
os.mkdir('./outputs/survival/multivariate_all_cancers/')
for state in states:
    print(state)
    
    cph_concat = {}
    cph_concat['OS'] = pd.DataFrame()
    cph_concat['PFS'] = pd.DataFrame()
    
    for cancer in cancers:
        print(cancer)
        ix = state_scores[cancer][state].index.values
        
        cph_dict = fit_cph(end1_name = 'OS', end1_time = TCGA_sample_info.loc[ix,'OS.time'], end1_event = TCGA_sample_info.loc[ix, 'OS'], 
                           end2_name = 'PFS', end2_time = TCGA_survival.loc[ix, 'PFS.time'], end2_event = TCGA_survival.loc[ix, 'PFS'], 
                           var1 = state_scores[cancer][state], var1_name = state, 
                           var2 = TCGA_sample_info.loc[ix, 'sex'], var2_name = 'Sex', 
                           #var3 =  TCGA_sample_info.loc[ix, 'age_at_initial_pathologic_diagnosis'], var3_name = 'Age', 
                           dummy_var1 = TCGA_sample_info.loc[ix,'tumor_stage'], dummy_var1_name = ('Stage (versus 0 & I)', 'Stage 0 & I'), 
                           advice = False, strata = ['Stage (versus 0 & I)'], drop = ['Sex'])
        
        for end in cph_dict.keys():
            print(end)
            print(cph_dict[end]['n'])
            print(cph_dict[end]['na'])
            
            cph_dict[end]['cph'].index = cancer + ' ' + cph_dict[end]['cph'].index 
            cph_concat[end] = pd.concat([cph_concat[end], cph_dict[end]['cph']])
        
    for end in cph_concat.keys():

        plot_forest_cph(cph_concat[end], title = end, xlabel = 'log10(HR) (95% CI)', sort_on_pvalues = True, nan_str = False, 
                        fig_adjust = (5, 1.25), save = './outputs/survival/multivariate_all_cancers/' + end + '_' + state +'.pdf')
    
    print('\n')
    print('\n')

CD8B
BRCA
Fitting: OS ...
Number of events: 140.0
Proportional hazard assumption looks okay.
Concordance index: 0.579
Finished fitting: OS!

Fitting: PFS ...
Number of events: 191.0
Proportional hazard assumption looks okay.
Concordance index: 0.568
Finished fitting: PFS!

OS
1070
{'CD8B': 0, 'Sex': 0, 'Stage (versus 0 & I)': 24, 'time': 1, 'event': 0}
PFS
1070
{'CD8B': 0, 'Sex': 0, 'Stage (versus 0 & I)': 24, 'time': 1, 'event': 0}
HNSC
Fitting: OS ...
Number of events: 189.0
Proportional hazard assumption looks okay.
Concordance index: 0.555
Finished fitting: OS!

Fitting: PFS ...
Number of events: 234.0
Proportional hazard assumption looks okay.
Concordance index: 0.551
Finished fitting: PFS!

OS
444
{'CD8B': 0, 'Sex': 0, 'Stage (versus 0 & I)': 75, 'time': 1, 'event': 0}
PFS
444
{'CD8B': 0, 'Sex': 0, 'Stage (versus 0 & I)': 75, 'time': 1, 'event': 0}
LUAD
Fitting: OS ...
Number of events: 181.0
Proportional hazard assumption looks okay.
Concordance index: 0.543
Finished fitting: OS

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 384 total observations, 237 right-censored observations>
         test_name = proportional_hazard_test

---
           test_statistic    p  -log2(p)
CD8B km              0.51 0.48      1.07
     rank            6.04 0.01      6.16
Sex  km              0.00 0.97      0.04
     rank            1.13 0.29      1.80



1. Variable 'CD8B' failed the non-proportional test: p-value is 0.0140.

Concordance index: 0.537
Finished fitting: OS!

Fitting: PFS ...
Number of events: 184.0
Proportional hazard assumption looks okay.
Concordance index: 0.55
Finished fitting: PFS!

OS
384
{'CD8B': 0, 'Sex': 0, 'Stage (versus 0 & I)': 25, 'time': 6, 'event': 0}
PFS
386
{'CD8B': 0, 'Sex': 0, 'Stage (versus 0 & I)': 25, 'time': 4, 'event': 0}
KIRC
Fitting: OS ...
Number of events: 174.0
Proportional hazard assumption looks okay.
Concordance index: 0.514
Finished fitting: OS!

Fitting: PFS ...
Number of events: 221.0
Proportional hazard assumption looks okay.
Concordance index: 0.524
Finished fitting: PFS!

OS
530
{'CD8B': 0, 'Sex': 0, 'Stage (versus 0 & I)': 3, 'time': 0, 'event': 0}
PFS
528
{'CD8B': 0, 'Sex': 0, 'Stage (versus 0 & I)': 3, 'time': 2, 'event': 0}
BLCA
Fitting: OS ...
Number of events: 177.0
Proportional hazard assumption looks okay.
Concordance index: 0.53
Finished fitting: OS!

Fitting: PFS ...
Numb

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 386 total observations, 202 right-censored observations>
         test_name = proportional_hazard_test

---
            test_statistic    p  -log2(p)
CCR10 km              1.09 0.30      1.75
      rank            7.72 0.01      7.51
Sex   km              0.00 0.97      0.05
      rank            2.66 0.10      3.28



1. Variable 'CCR10' failed the non-proportional test: p-value is 0.0055.

Concordance index: 0.58
Finished fitting: PFS!

OS
384
{'CCR10': 0, 'Sex': 0, 'Stage (versus 0 & I)': 25, 'time': 6, 'event': 0}
PFS
386
{'CCR10': 0, 'Sex': 0, 'Stage (versus 0 & I)': 25, 'time': 4, 'event': 0}
KIRC
Fitting: OS ...
Number of events: 174.0
Proportional hazard assumption looks okay.
Concordance index: 0.56
Finished fitting: OS!

Fitting: PFS ...
Number of events: 221.0
Proportional hazard assumption looks okay.
Concordance index: 0.542
Finished fitting: PFS!

OS
530
{'CCR10': 0, 'Sex': 0, 'Stage (versus 0 & I)': 3, 'time': 0, 'event': 0}
PFS
528
{'CCR10': 0, 'Sex': 0, 'Stage (versus 0 & I)': 3, 'time': 2, 'event': 0}
BLCA
Fitting: OS ...
Number of events: 177.0


...



1. Variable 'CCR10' failed the non-proportional test: p-value is 0.0002.

Concordance index: 0.539
Finished fitting: PFS!

OS
405
{'CCR10': 0, 'Sex': 0, 'Stage (versus 0 & I)': 2, 'time': 1, 'event': 0}
PFS
406
{'CCR10': 0, 'Sex': 0, 'Stage (versus 0 & I)': 2, 'time': 0, 'event': 0}
LIHC
Fitting: OS ...
Number of events: 116.0
Proportional hazard assumption looks okay.
Concordance index: 0.551
Finished fitting: OS!

Fitting: PFS ...
Number of events: 208.0
Proportional hazard assumption looks okay.
Concordance index: 0.527
Finished fitting: PFS!

OS
346
{'CCR10': 0, 'Sex': 0, 'Stage (versus 0 & I)': 24, 'time': 1, 'event': 0}
PFS
346
{'CCR10': 0, 'Sex': 0, 'Stage (versus 0 & I)': 24, 'time': 1, 'event': 0}
SKCM
Fitting: OS ...
Number of events: 189.0
Proportional hazard assumption looks okay.
Concordance index: 0.556
Finished fitting: OS!

Fitting: PFS ...
Number of events: 287.0
Proportional hazard assumption looks okay.
Concordance index: 0.525
Finished fitting: PFS!

OS
408
{'CCR1

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 1070 total observations, 879 right-censored observations>
         test_name = proportional_hazard_test

---
             test_statistic    p  -log2(p)
CXCL16 km              7.03 0.01      6.96
       rank            0.12 0.73      0.46
Sex    km              0.38 0.54      0.90
       rank            1.59 0.21      2.27



1. Variable 'CXCL16' failed the non-proportional test: p-value is 0.0080.

Concordance index: 0.518
Finished fitting: PFS!

OS
1070
{'CXCL16': 0, 'Sex': 0, 'Stage (versus 0 & I)': 24, 'time': 1, 'event': 0}
PFS
1070
{'CXCL16': 0, 'Sex': 0, 'Stage (versus 0 & I)': 24, 'time': 1, 'event': 0}
HNSC
Fitting: OS ...
Number of events: 189.0
Proportional hazard assumption looks okay.
Concordance index: 0.526
Finished fitting: OS!

Fitting: PFS ...
Number of events: 234.0
Proportional hazard assumption looks okay.
Concordance index: 0.54
Finished fitting: PFS!

OS
444
{'CXCL16': 0, 'Sex': 0, 'Stage (versus 0 & I)': 75, 'time': 1, 'event': 0}
PFS
444
{'CXCL16': 0, 'Sex': 0, 'Stage (versus 0 & I)': 75, 'time': 1, 'event': 0}
LUAD
Fitting: OS ...
Number of events: 181.0
Proportional hazard assumption looks okay.
Concordance index: 0.562
Finished fitting: OS!

Fitting: PFS ...
Number of events: 256.0


<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 498 total observations, 242 right-censored observations>
         test_name = proportional_hazard_test

---
             test_statistic    p  -log2(p)
CXCL16 km              6.73 0.01      6.72
       rank            3.07 0.08      3.65
Sex    km              3.01 0.08      3.60
       rank            0.08 0.78      0.36



1. Variable 'CXCL16' failed the non-proportional test: p-value is 0.0095.

Concordance index: 0.559
Finished fitting: PFS!

OS
498
{'CXCL16': 0, 'Sex': 0, 'Stage (versus 0 & I)': 8, 'time': 9, 'event': 0}
PFS
498
{'CXCL16': 0, 'Sex': 0, 'Stage (versus 0 & I)': 8, 'time': 9, 'event': 0}
LUSC
Fitting: OS ...
Number of events: 210.0
Proportional hazard assumption looks okay.
Concordance index: 0.548
Finished fitting: OS!

Fitting: PFS ...
Number of events: 250.0
Proportional hazard assumption looks okay.
Concordance index: 0.527
Finished fitting: PFS!

OS
491
{'CXCL16': 0, 'Sex': 0, 'Stage (versus 0 & I)': 4, 'time': 6, 'event': 0}
PFS
492
{'CXCL16': 0, 'Sex': 0, 'Stage (versus 0 & I)': 4, 'time': 5, 'event': 0}
STAD
Fitting: OS ...
Number of events: 147.0
Proportional hazard assumption looks okay.
Concordance index: 0.514
Finished fitting: OS!

Fitting: PFS ...
Number of events: 184.0
Proportional hazard assumption looks okay.
Concordance index: 0.536
Finished fitting: PFS!

OS
384
{'C

## Kaplan Meiers

In [27]:
states = ['CD8B', 'CXCR6', 'CXCR3', 'CXCR2', 'CCR5', 'CCR10', 'IL12B', 'CXCL16', 'NK cells', 'CD19']

In [28]:
quantile = 1/3
for cancer in cancers:
    print(cancer)
    os.mkdir('./outputs/survival/' + cancer + '/kaplan_meier/')
    # dichotomize patients into botom 1/3 and top 1/3 groups using the state signature scores: 
    state_scores_0 = {}
    state_scores_66 = {}

    for state in states: 
        cmask = (state_scores[cancer][state] <= state_scores[cancer][state].quantile(q = quantile))
        state_scores_0[state] = state_scores[cancer][state][cmask]
        print('Number of patients in 0_33 group '+ state + ':', len(state_scores_0[state]))
        print(state_scores_0[state].min(), state_scores_0[state].max(), '\n')
    
        cmask = (state_scores[cancer][state] >= state_scores[cancer][state].quantile(q = 2*quantile))
        state_scores_66[state] = state_scores[cancer][state][cmask]
        print('Number of patients in 67-100 group '+ state + ':', len(state_scores_66[state]))
        print(state_scores_66[state].min(), state_scores_66[state].max(), '\n')
    
    # Extract OS_time/event and PFS_time/event: 

    OS_time = TCGA_sample_info[TCGA_sample_info['type']==cancer]['OS.time']
    OS_status = TCGA_sample_info[TCGA_sample_info['type']==cancer]['OS']
    PFS_time =TCGA_survival[TCGA_survival['type']==cancer]['PFS.time']
    PFS_status = TCGA_survival[TCGA_survival['type']==cancer]['PFS']

    # Fit KaplanMeier PFS and plot:  
    for state in states:
        print('PFS: ', state)
        ix0 = state_scores_0[state].index.values 
        ix67 = state_scores_66[state].index.values 
        
        cph_test_0 = pd.DataFrame({'PFS_time': PFS_time[ix0].dropna()/365, 
                                   'ix': 0, 
                                   'PFS_event': PFS_status[PFS_time[ix0].dropna().index.values].dropna()}) # here you can do the last .dropna() because you fill ix with dummy 0 
        
        cph_test_67 = pd.DataFrame({'PFS_time': PFS_time[ix67].dropna()/365, 
                                    'ix': 1, 
                                    'PFS_event': PFS_status[PFS_time[ix67].dropna().index.values].dropna()}) # here you can do the last.dropna() because you fill ix with dummy 1
        
        print('Number of events ix0:', PFS_status[PFS_time[ix0].dropna().index.values].dropna().sum(), 'Number of missing values ix0:', PFS_status[PFS_time[ix0].dropna().index.values].isna().sum())
        print('Number of events ix67:', PFS_status[PFS_time[ix67].dropna().index.values].dropna().sum(), 'Number of missing values ix67:', PFS_status[PFS_time[ix67].dropna().index.values].isna().sum())
        
        cph_test = pd.concat([cph_test_0, cph_test_67]) 
        
        cph = CoxPHFitter()
        cph.fit(cph_test, duration_col='PFS_time', event_col='PFS_event')
        cph.check_assumptions(cph_test)
        
        kmf_0 = KaplanMeierFitter() 
        kmf_67 = KaplanMeierFitter()

        kmf_0.fit(PFS_time[ix0].dropna()/365, 
                  PFS_status[PFS_time[ix0].dropna().index.values].dropna(), 
                  timeline = np.arange(0, 10, 0.005),
                  label = 'Low (n=' + str(len(PFS_time[ix0].dropna())) + ')')
        ax = kmf_0.plot(ci_show = False, color = 'black', show_censors = True, censor_styles={'marker': 2, 'ms':5, 'lw':0.5})

        
        kmf_67.fit(PFS_time[ix67].dropna()/365, 
                   PFS_status[PFS_time[ix67].dropna().index.values].dropna(),
                   timeline = np.arange(0, 10, 0.005),
                   label = 'High (n=' + str(len(PFS_time[ix67].dropna())) + ')')
        ax = kmf_67.plot(ci_show = False, color = 'red', show_censors = True, censor_styles={'marker': 2, 'ms':5, 'lw':0.5})
        
        ax.text(0.01, 0.01, 'HR '+str(round(cph.summary.loc['ix', 'exp(coef)'], 2))+' ('+str(round(cph.summary.loc['ix', 'exp(coef) lower 95%'],2))+'-'+str(round(cph.summary.loc['ix', 'exp(coef) upper 95%'],2))+')'+'\n'+'p='+str(round(cph.summary.loc['ix', 'p'], 10)),
                ha='left', va='bottom', transform=ax.transAxes)
            
        #add_at_risk_counts(kmf_0, kmf_67, ax=ax)
        plt.title(cancer + ' PFS ' + state)
        plt.tight_layout()
        plt.savefig('./outputs/survival/' + cancer + '/kaplan_meier/PFS_km_33_' + state + '.pdf', dpi = 400)
        plt.close()
    
    # to make OS and PFS completely separate initialize a new for loop: 
    for state in states:
        print('OS: ', state)
        ix0 = state_scores_0[state].index.values 
        ix67 = state_scores_66[state].index.values 
        
        cph_test_0 = pd.DataFrame({'OS_time': OS_time[ix0].dropna()/365, 
                                    'ix': 0, 
                                    'OS_event': OS_status[OS_time[ix0].dropna().index.values].dropna()}) # here you can do the last .dropna() because you fill ix with dummy 0 
        
        cph_test_67 = pd.DataFrame({'OS_time': OS_time[ix67].dropna()/365, 
                                    'ix': 1, 
                                    'OS_event': OS_status[OS_time[ix67].dropna().index.values].dropna()}) # here you can do the last .dropna() because you fill ix with dummy 1 
        
        print('Number of events ix0:', OS_status[OS_time[ix0].dropna().index.values].dropna().sum(), 'Number of missing values ix0:', OS_status[OS_time[ix0].dropna().index.values].isna().sum())
        print('Number of events ix67:', OS_status[OS_time[ix67].dropna().index.values].dropna().sum(), 'Number of missing values ix67:', OS_status[OS_time[ix67].dropna().index.values].isna().sum())
        
        cph_test = pd.concat([cph_test_0, cph_test_67]) 
        
        cph = CoxPHFitter()
        cph.fit(cph_test, duration_col='OS_time', event_col='OS_event')
        cph.check_assumptions(cph_test)
        
        kmf_0 = KaplanMeierFitter() 
        kmf_67 = KaplanMeierFitter()

        kmf_0.fit(OS_time[ix0].dropna()/365, 
                  OS_status[OS_time[ix0].dropna().index.values].dropna(),
                  timeline = np.arange(0, 10, 0.005),
                  label = 'Low (n=' + str(len(OS_time[ix0].dropna())) + ')')
        ax = kmf_0.plot(ci_show = False, color = 'black', show_censors = True, censor_styles={'marker': 2, 'ms':5, 'lw':0.5})
        
        kmf_67.fit(OS_time[ix67].dropna()/365, 
                   OS_status[OS_time[ix67].dropna().index.values].dropna(),
                   timeline = np.arange(0, 10, 0.005),
                   label = 'High (n=' + str(len(OS_time[ix67].dropna())) + ')')
        ax = kmf_67.plot(ci_show = False, color = 'red', show_censors = True, censor_styles={'marker': 2, 'ms':5, 'lw':0.5})
        
        ax.text(0.01, 0.01, 'HR '+str(round(cph.summary.loc['ix', 'exp(coef)'], 2))+' ('+str(round(cph.summary.loc['ix', 'exp(coef) lower 95%'],2))+'-'+str(round(cph.summary.loc['ix', 'exp(coef) upper 95%'],2))+')'+'\n'+'p='+str(round(cph.summary.loc['ix', 'p'], 10)),
                ha='left', va='bottom', transform=ax.transAxes)
        
        #add_at_risk_counts(kmf_0, kmf_67, ax=ax)
        plt.title(cancer + ' OS ' + state)
        plt.tight_layout()
        plt.savefig('./outputs/survival/' + cancer + '/kaplan_meier/OS_km_33_' + state + '.pdf', dpi = 400)
        plt.close()
        
    print('\n')
    print('\n')

BRCA
Number of patients in 0_33 group CD8B: 365
-3.36906952828317 -0.3797748698479904 

Number of patients in 67-100 group CD8B: 365
0.46397804076015414 2.925674533481078 

Number of patients in 0_33 group CXCR6: 365
-3.315627990370191 -0.4318423731465461 

Number of patients in 67-100 group CXCR6: 365
0.4596873084543077 2.7074374084166895 

Number of patients in 0_33 group CXCR3: 365
-3.5641891989573553 -0.38450408689199633 

Number of patients in 67-100 group CXCR3: 365
0.4068573531684912 2.510956860210585 

Number of patients in 0_33 group CXCR2: 365
-3.1549186452901568 -0.3048504812409999 

Number of patients in 67-100 group CXCR2: 365
0.42487742543984863 5.069117017843321 

Number of patients in 0_33 group CCR5: 365
-4.024744375427019 -0.4232818580538839 

Number of patients in 67-100 group CCR5: 365
0.46663258730890633 2.8339548726892962 

Number of patients in 0_33 group CCR10: 365
-3.235943817821323 -0.4568297614025395 

Number of patients in 67-100 group CCR10: 365
0.398161321

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 347 total observations, 195 right-censored observations>
         test_name = proportional_hazard_test

---
         test_statistic    p  -log2(p)
ix km              4.75 0.03      5.10
   rank            6.69 0.01      6.68



1. Variable 'ix' failed the non-proportional test: p-value is 0.0097.

   Advice: with so few unique values (only 2), you can include `strata=['ix', ...]` in the call in
`.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification





LUAD
Number of patients in 0_33 group CD8B: 172
-3.2248854660298383 -0.4277101905591369 

Number of patients in 67-100 group CD8B: 172


<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 338 total observations, 162 right-censored observations>
         test_name = proportional_hazard_test

---
         test_statistic      p  -log2(p)
ix km              9.86 <0.005      9.21
   rank            9.26 <0.005      8.74



1. Variable 'ix' failed the non-proportional test: p-value is 0.0017.

   Advice: with so few unique values (only 2), you can include `strata=['ix', ...]` in the call in
`.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification

PFS:  CCR10
Number of events ix0: 85.0 Number of missing values ix0: 0
Number of events ix67: 83.0 Number of missing values ix67: 0
Proport

...



1. Variable 'ix' failed the non-proportional test: p-value is 0.0018.

   Advice: with so few unique values (only 2), you can include `strata=['ix', ...]` in the call in
`.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification

OS:  CD8B
Number of events ix0: 68.0 Number of missing values ix0: 0
Number of events ix67: 59.0 Number of missing values ix67: 0
Proportio

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 337 total observations, 207 right-censored observations>
         test_name = proportional_hazard_test

---
         test_statistic      p  -log2(p)
ix km              7.92 <0.005      7.68
   rank            5.44   0.02      5.67



1. Variable 'ix' failed the non-proportional test: p-value is 0.0049.

   Advice: with so few unique values (only 2), you can include `strata=['ix', ...]` in the call in
`.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification

OS:  CXCL16
Number of events ix0: 69.0 Number of missing values ix0: 0
Number of events ix67: 54.0 Number of missing values ix67: 0
Proport

<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 272 total observations, 124 right-censored observations>
         test_name = proportional_hazard_test

---
         test_statistic    p  -log2(p)
ix km              5.26 0.02      5.52
   rank            5.92 0.01      6.06



1. Variable 'ix' failed the non-proportional test: p-value is 0.0149.

   Advice: with so few unique values (only 2), you can include `strata=['ix', ...]` in the call in
`.fit`. See documentation in link [E] below.

---
[A]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html
[B]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Bin-variable-and-stratify-on-it
[C]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Introduce-time-varying-covariates
[D]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Modify-the-functional-form
[E]  https://lifelines.readthedocs.io/en/latest/jupyter_notebooks/Proportional%20hazard%20assumption.html#Stratification

PFS:  CCR5
Number of events ix0: 73.0 Number of missing values ix0: 0
Number of events ix67: 77.0 Number of missing values ix67: 0
Proporti